In [39]:
import pandas as pd

# 读取数据
financial_ratio = pd.read_csv('/Users/shawn/Github/M1/金融機構與風險管理/S_P500 財務比率_rename_V2.csv')
rating = pd.read_csv('/Users/shawn/Github/M1/金融機構與風險管理/rating.csv')

financial_ratio['datadate'] = pd.to_datetime(financial_ratio['datadate'], format='%Y/%m/%d')
financial_ratio['datadate'] = financial_ratio['datadate'].dt.strftime('%Y-%m-%d')


rating['datadate'] = pd.to_datetime(rating['datadate'], format='%Y-%m-%d')
rating['datadate'] = rating['datadate'].dt.strftime('%Y-%m-%d')

In [40]:
merged_df = pd.merge(financial_ratio, rating[['datadate', 'ticker', 'splticrm']], on=['datadate', 'ticker'], how='left')
# merged_df['dividend yield'] = merged_df['dividend yield'].str.rstrip('%').astype('float') / 100


print(merged_df[merged_df['ticker'] == 'AMD'][['datadate','ticker','splticrm']].head()) 
print(rating[rating['ticker'] == 'AMD'][['datadate','ticker','splticrm']].head()) 

        datadate ticker splticrm
3137  2010-01-31    AMD       B-
3138  2010-02-28    AMD       B-
3139  2010-03-31    AMD       B-
3140  2010-04-30    AMD       B-
3141  2010-05-31    AMD       B-
       datadate ticker splticrm
388  2001-01-31    AMD        B
389  2001-02-28    AMD        B
390  2001-03-31    AMD        B
391  2001-04-30    AMD        B
392  2001-05-31    AMD        B


In [41]:
# 以插值法填入
def fill_missing_values_with_interpolation(df, group_col):
    df_filled = df.groupby(group_col).apply(
        lambda group: group.interpolate(method='linear').round(3)
    ).reset_index(drop=True)
    return df_filled
merged_df_filled = fill_missing_values_with_interpolation(merged_df, 'ticker')
merged_df_filled.to_csv('/Users/shawn/Github/M1/金融機構與風險管理/filled_test.csv')

In [42]:
# financial_ratio, rating期間範圍無對齊
merged_df.dropna(subset=['splticrm'], inplace=True)
def filter_rows(group):
    # 組內rating前後不同的才會被留下
    mask = group['splticrm'] != group['splticrm'].shift()
    return group[mask]
merged_df = merged_df.groupby('ticker').apply(filter_rows).reset_index(drop=True)
merged_df['ticker'].value_counts().to_frame() 


,count
ticker,
CHK,12
AKS,7
ETFC,7
M,7
DO,7
...,...
JNJ,1
KEY,1
KMB,1


In [43]:
# merged_df保留了所有ticker
merged_df2 = merged_df.groupby('ticker').filter(lambda x: len(x) > 1)
print(merged_df2['ticker'].value_counts().to_frame())
print(len(merged_df2))
# merged_df2.to_csv('SP500_merge.csv')
merged_df2.head()

        count
ticker       
CHK        12
ETFC        7
DO          7
AKS         7
M           7
...       ...
OKE         2
D           2
ODP         2
DD          2
PEG         2

[225 rows x 1 columns]
668


,datadate,ticker,after-tax interest coverage,interest coverage ratio,cash flow/ total debt,operating CF/ current liabilities,operating margin before dep.,operating margin after dep.,return on equity,return on capital employed,...,pre-tax return on net operating assets,pre-tax return on total earning assets,gross profit/ total assets,common equity/invested capital,LTD/ invested capital,total debt/ invested capital,capitalization ratio,cash flow margin,dividend yield,splticrm
1,2010-01-31,ABT,11.785,13.248,0.294,0.586,0.295,0.230,0.300,0.211,...,0.504,0.276,0.422,0.638,0.361,0.496,0.361,0.253,0.0302,AA
2,2012-10-31,ABT,9.919,16.970,0.251,0.540,0.303,0.208,0.106,0.152,...,0.394,0.199,0.343,0.667,0.330,0.451,0.331,0.254,0.0311,A+
3,2017-01-31,ABT,5.940,17.428,0.149,0.331,0.238,0.169,0.047,0.118,...,0.321,0.178,0.319,0.774,0.221,0.325,0.222,0.116,0.0254,BBB
4,2010-01-31,ADBE,114.445,214.785,0.467,1.324,0.337,0.248,0.080,0.137,...,0.340,0.248,0.381,0.830,0.170,0.170,0.170,0.227,NaN,BBB+
5,2013-12-31,ADBE,7.617,9.645,0.453,1.022,0.233,0.156,0.064,0.079,...,0.175,0.131,0.381,0.819,0.181,0.183,0.181,0.184,NaN,A-


In [44]:
ratings_map = {
    'D': 0, 'CC': 1, 'CCC': 2, 'CCC+': 3,
    'B-': 5, 'B': 6, 'B+': 7, 'BB-': 8, 'BB': 9, 'BB+': 10,
    'BBB-': 12, 'BBB': 14, 'BBB+': 16,
    'A-': 19, 'A': 22, 'A+': 25, 'AA-': 28, 'AA': 32, 'AA+': 36, 'AAA': 40
}

# 將評級參數化
merged_df2['rating'] = merged_df2['splticrm'].map(ratings_map)
merged_df2.head()


,datadate,ticker,after-tax interest coverage,interest coverage ratio,cash flow/ total debt,operating CF/ current liabilities,operating margin before dep.,operating margin after dep.,return on equity,return on capital employed,...,pre-tax return on total earning assets,gross profit/ total assets,common equity/invested capital,LTD/ invested capital,total debt/ invested capital,capitalization ratio,cash flow margin,dividend yield,splticrm,rating
1,2010-01-31,ABT,11.785,13.248,0.294,0.586,0.295,0.230,0.300,0.211,...,0.276,0.422,0.638,0.361,0.496,0.361,0.253,0.0302,AA,32
2,2012-10-31,ABT,9.919,16.970,0.251,0.540,0.303,0.208,0.106,0.152,...,0.199,0.343,0.667,0.330,0.451,0.331,0.254,0.0311,A+,25
3,2017-01-31,ABT,5.940,17.428,0.149,0.331,0.238,0.169,0.047,0.118,...,0.178,0.319,0.774,0.221,0.325,0.222,0.116,0.0254,BBB,14
4,2010-01-31,ADBE,114.445,214.785,0.467,1.324,0.337,0.248,0.080,0.137,...,0.248,0.381,0.830,0.170,0.170,0.170,0.227,NaN,BBB+,16
5,2013-12-31,ADBE,7.617,9.645,0.453,1.022,0.233,0.156,0.064,0.079,...,0.131,0.381,0.819,0.181,0.183,0.181,0.184,NaN,A-,19


In [45]:
def process_data(group):
    # 保留前一期的評價
    group['prev_rating'] = group['rating'].shift(1)
    group['rating_diff'] = group['rating'] - group['rating'].shift(1)
    # 變化率
    for col in group.columns:
        if pd.api.types.is_numeric_dtype(group[col]):
            group[col] = group[col].pct_change()
            # group[col + '_change'] = group[col].pct_change()

    # 刪除前一期評價(prev_rating)
    # group = group[group['is_upgraded'] == 1].drop(columns=['prev_rating'])
    group = group.dropna(subset=['prev_rating'])

    return group

In [46]:
merged_df3 = merged_df2.groupby('ticker').apply(process_data).reset_index(drop=True)
merged_df3[['ticker','prev_rating','splticrm','rating_diff']].head()
merged_df3.head()

,datadate,ticker,after-tax interest coverage,interest coverage ratio,cash flow/ total debt,operating CF/ current liabilities,operating margin before dep.,operating margin after dep.,return on equity,return on capital employed,...,common equity/invested capital,LTD/ invested capital,total debt/ invested capital,capitalization ratio,cash flow margin,dividend yield,splticrm,rating,prev_rating,rating_diff
0,2017-01-31,ABT,-0.401149,0.026989,-0.406375,-0.387037,-0.214521,-0.187500,-0.556604,-0.223684,...,0.160420,-0.330303,-0.279379,-0.329305,-0.543307,-0.183280,BBB,-0.440000,-0.218750,0.571429
1,2016-07-31,ADI,-0.733025,-0.688609,-0.076923,-0.386531,0.055233,0.013514,-0.436364,-0.332000,...,-0.069397,0.504132,0.760331,0.504132,-0.089606,-0.060714,BBB,-0.263158,0.187500,-2.666667
2,2013-12-31,AEE,-2.895230,-0.020243,0.000000,-0.387940,0.025974,0.000000,-1.500000,-0.096774,...,0.060729,-0.062626,-0.011516,-0.062626,-1.789474,-0.032823,BBB+,0.142857,0.166667,0.000000
3,2017-02-28,AEP,-0.439189,0.031147,-0.109091,-0.197302,0.018405,0.065327,-0.637931,0.072289,...,-0.015873,0.014113,0.089041,0.014113,-0.316872,0.008596,A-,0.187500,0.142857,0.500000
4,2011-05-31,AES,0.085962,0.085447,0.625000,0.133333,0.020690,0.032558,0.220339,0.022901,...,0.101382,-0.033019,-0.009859,-0.034853,-0.189349,NaN,BB-,-0.111111,0.125000,-2.000000


In [47]:
# merged_df3 = merged_df3.drop(columns=['gvkey','permno','adate','qdate','rating_diff_change','prev_rating_change','rating_change'])
merged_df3.to_csv('/Users/shawn/Github/M1/金融機構與風險管理/SP500_change_V5_rename.csv')

In [9]:
merged_df3[['splticrm','rating','prev_rating','rating_diff']]

,splticrm,rating,prev_rating,rating_diff
0,A+,25,32.0,-7.0
1,BBB,14,25.0,-11.0
2,A-,19,16.0,3.0
3,A-,19,16.0,3.0
4,BBB,14,19.0,-5.0
...,...,...,...,...
438,A-,19,16.0,3.0
439,BBB+,16,19.0,-3.0
440,BBB-,12,14.0,-2.0
441,BBB,14,12.0,2.0
